## 1. Connect to Azure OpenAI LLM
Initialize and connect to the Azure OpenAI service using credentials from environment variables to enable chat completions.

In [1]:
import os
import dotenv


In [2]:
dotenv.load_dotenv()

True

In [3]:
#connect to llm
from openai import AzureOpenAI
from dotenv import load_dotenv


client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    azure_endpoint=os.getenv("OPENAI_BASE_URL"),
    api_version="2024-08-01-preview"
)

response = client.chat.completions.create(   
  model="gpt-4o",#  Replace with your actual deployment name from Azure Portal
  messages=[
    {"role": "user", "content": "This is a test."}
  ]
)

print(response.model_dump_json(indent=2))

{
  "id": "chatcmpl-Cv6kuO6T4qeapDWKQX6ImzlU6O3lH",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Got it! How can I assist you today?",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "protected_material_code": {
          "filtered": false,
          "detected": false
        },
        "protected_material_text": {
          "filtered": false,
          "detected": false
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
    

## 2. Install Azure Search Documents Library
Install the required `azure-search-documents` package to interact with Azure AI Search service.
- pip install azure-search-documents

## 3. Connect to Azure AI Search Service
Initialize the SearchClient with endpoint, index name, and API key from environment variables to enable queries against the Azure Search index.

In [4]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

service_endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
index_name = os.environ["AZURE_SEARCH_INDEX_NAME"]
key = os.environ["AZURE_SEARCH_API_KEY"]

search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))

## 4. Search and Display Results
Execute a search query against the Azure Search index and display all results with their fields and values.

In [ ]:
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType

# Semantic search with reranking
results = search_client.search(
    search_text="ROBERT AUDI",
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name="default",  # or your configured semantic config name
    top=5  # number of results to return
)

print("Semantic Search Results...")
result_list = list(results)
print(f"Number of results: {len(result_list)}")

if result_list:
    print("\nFirst result:")
    print(result_list[0])
    
    for i, result in enumerate(result_list):
        print(f"\n--- Result {i+1} ---")
        for key, value in result.items():
            print(f"{key}: {value}")
else:
    print("No results found")

## 8. Compare Normal Search vs Semantic Search

### What You'll See

This cell performs **both search types** on the same query and compares the results:

- **🔵 Normal Keyword Search**: Matches exact words in your documents
- **🟢 Semantic Search**: Understands meaning and context, ranks by relevance

### Which One is Better?

It depends on your use case:
- **Normal Search**: Fast, good for exact matches ("John Smith")
- **Semantic Search**: Smarter, better at understanding intent ("Who is the main character?")

**For RAG applications**: Semantic Search usually gives better results because it understands meaning, not just keywords.

### Try It!

Enter a search query below and compare the results from both search methods.

In [ ]:
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType

# Define which fields you want to display
# those fields and data just for test that's why the ain't that good 
IMPORTANT_FIELDS = [
    'metadata_storage_name',      # File name
    'content',                     # Main content
    'keyphrases',                  # Key phrases
    'people',                       # People mentioned
]

user_query = input("What do you want to search for? = ")

print(f"\n{'='*70}")
print(f"QUERY: {user_query}")
print(f"{'='*70}\n")

# ============================================================================
# 1. NORMAL (KEYWORD) SEARCH
# ============================================================================
print("🔵 NORMAL KEYWORD SEARCH")
print("-" * 70)

normal_results = search_client.search(search_text=user_query, top=5)
normal_list = list(normal_results)

print(f"Results found: {len(normal_list)}\n")

if normal_list:
    for i, result in enumerate(normal_list, 1):
        print(f"  Result {i}:")
        for key, value in result.items():
            # Only show important fields
            if key in IMPORTANT_FIELDS:
                # Truncate long values for readability
                if isinstance(value, str):
                    value_str = value[:150] + "..." if len(value) > 150 else value
                else:
                    value_str = str(value)[:150] + "..." if len(str(value)) > 150 else str(value)
                print(f"    {key}: {value_str}")
        print()
else:
    print("  No results found.\n")

# ============================================================================
# 2. SEMANTIC SEARCH
# ============================================================================
print("\n" + "="*70)
print("🟢 SEMANTIC SEARCH (AI-Powered Ranking)")
print("-" * 70)

semantic_results = search_client.search(
    search_text=user_query,
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name="default",
    top=5
)

semantic_list = list(semantic_results)

print(f"Results found: {len(semantic_list)}\n")

if semantic_list:
    for i, result in enumerate(semantic_list, 1):
        print(f"  Result {i}:")
        for key, value in result.items():
            # Only show important fields
            if key in IMPORTANT_FIELDS:
                # Truncate long values for readability
                if isinstance(value, str):
                    value_str = value[:150] + "..." if len(value) > 150 else value
                else:
                    value_str = str(value)[:150] + "..." if len(str(value)) > 150 else str(value)
                print(f"    {key}: {value_str}")
        print()
else:
    print("  No results found.\n")

# ============================================================================
# 3. COMPARISON
# ============================================================================
print("\n" + "="*70)
print("📊 COMPARISON")
print("-" * 70)
print(f"Normal Search Results:    {len(normal_list)} documents")
print(f"Semantic Search Results:  {len(semantic_list)} documents")
print("\n💡 Note:")
print("  - Normal Search: Matches keywords in documents")
print("  - Semantic Search: Understands meaning and context better")
print("="*70 + "\n")

# single turn-chat


In [ ]:
#okay we will be using the semantic search for the rag 
#here we will just retrive then pass directly to the agent to reformulate the answer and its gonna be single-turn chat 
# then we will do muilt-turn-chat and see the result with query rewriting  

user_message = input("write here you message = ")
#let's retrive somthing cool 
semantic_results = search_client.search(
  search_text=user_message,
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name="default",
    select=[ "keyphrases"],  # Exclude 'content'
    top=1
)

semantic_list = list(semantic_results)
#The top=1 parameter specifies how many search results you want returned from Azure AI Search.


In [ ]:
"""For RAG applications, you typically want to retrieve multiple relevant documents (not just 1) to give the LLM more context. Common values are:"""

In [ ]:
"""
For RAG applications, you should convert the search results to a string before passing them to the LLM. Here's why and how:

Why String Format?
LLMs work with text input, not Python objects
You need to structure the context in a readable format
You want to include only relevant fields (not all metadata)

"""
#i think of using MS markitdown 


In [13]:
def to_markdown(results):## turn the search to markdown 
    if not results:
        return "No relevant documents found."
    blocks = []
    for i, doc in enumerate(results, 1):
        blocks.append(
            f"### Document {i}\n"
            f"- **filename:** {doc.get('metadata_storage_name', 'N/A')}\n"
            f"- **people:** {', '.join(doc.get('people', [])) if doc.get('people') else 'N/A'}\n"
            f"- **keyphrases:** {', '.join(doc.get('keyphrases', [])) if doc.get('keyphrases') else 'N/A'}\n\n"
            f"**content:**\n{doc.get('content', 'N/A')}\n"
        )
    return "\n---\n".join(blocks)



In [15]:
# Adjust max_content_length based on your needs:
# - 2000 chars = ~500 tokens (very concise)
# - 4000 chars = ~1000 tokens (good balance)
# - 8000 chars = ~2000 tokens (more context)

context_md = to_markdown(semantic_list)
print(f"Context length: {len(context_md)} characters (~{len(context_md)//4} tokens)")

Context length: 34457 characters (~8614 tokens)


In [16]:
system_message= """ you are going to, revice a result of search from azure ai search index your job is to take that 
result and reforumlate an answer to the user based on his question 
"""
print((context_md))
#here is another error 
# i exceeded the rate limit 308373   this is the context you are planing to pass your model to 
#too much 

### Document 1
- **filename:** N/A
- **people:** N/A
- **keyphrases:** Charles J. Mach Distinguished Professor, CAMBRIDGE UNIVERSITY PRESS  T H E, Princeton University Allen W. Wood, Helsinki Bas van Fraassen, relevant collective licensing agreements, N D  E D, PHILOSOPHY S E C, T. H. Irwin, Arthur W. Burks, J. R. Lucas, D. M. Armstrong, Arthur C. Danto, Michael S. Moore, Martha C. Nussbaum, William P. Alston, Hector-Neri Castañeda, Roderick M. Chisholm, Patricia Smith Churchland, William L. Harper, São Paulo, Onora O’Neill, John R. Searle, The Edinburgh Building, Dagfinn Føllesdal, Latin American philosophy, third-party internet websites, accessible one-volume dictionary, New York University, O F, THE CAMBRIDGE, John Perry, 400 new entries, European philosophy, Continental philosophy, SECOND EDITION, ROBERT AUDI, Chinese, Italian, date survey, international team, Key features, comprehensive entries, major philosophers, Extensive coverage, developing fields, philoso- phy, applied ethic

In [17]:
# Send to LLM with proper system message and structured prompt
"""About the 429 error: This is a rate limit from Azure OpenAI - you've made too many requests. Just wait 30-60 seconds before running the cell again. This is not a code issue, it's Azure throttling your requests."""
response = client.chat.completions.create(   
  model="gpt-4o",
  messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "content": f"Context:\n{context_md}\n\nQuestion: {user_message}"}
  ]
)

print(response.choices[0].message.content)

It seems your query "robert" corresponds to the search result mentioning "Robert Audi" and might be related to his work or contributions within philosophy. Robert Audi is a highly regarded philosopher known for his works in epistemology, ethics, and the philosophy of religion. If you're looking for specific details about his contributions, major works, or aspects of his philosophical theories, please refine your query for further assistance.
